![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


In [2]:
SELECT * FROM companies


,company_id,company,city,country,continent
0,189,Otto Bock HealthCare,Duderstadt,Germany,Europe
1,848,Matrixport,,Singapore,Asia
2,556,Cloudinary,Santa Clara,United States,North America
3,999,PLACE,Bellingham,United States,North America
4,396,candy.com,New York,United States,North America
...,...,...,...,...,...
1069,1002,Poizon,Shanghai,China,Asia
1070,215,SSENSE,Montreal,Canada,North America
1071,866,Assembly,Culver City,United States,North America
1072,651,Xiaobing,Beijing,China,Asia


In [3]:
WITH top_industries AS 
(
	SELECT i.industry,
		COUNT(i.*)
	FROM industries AS i
	INNER JOIN dates AS d 
		ON i.company_id = d.company_id
	WHERE EXTRACT(year FROM d.date_joined) in ('2019','2020','2021')
	GROUP BY industry 
	ORDER BY count DESC
	LIMIT 3), 
	
	yearly_rankings AS 
	(
		SELECT COUNT(i.*) AS num_unicorns,i.industry ,
		EXTRACT(year FROM d.date_joined) AS year,
		AVG(f.valuation) AS average_valuation
	FROM industries AS i
	INNER JOIN dates AS d 
		ON i.company_id = d.company_id 
	INNER JOIN funding AS f 
		ON d.company_id = f.company_id 
	GROUP BY industry, year
		
)

SELECT industry,year, num_unicorns, ROUND(AVG(average_valuation / 1000000000 ), 2) AS average_valuation_billions
FROM yearly_rankings
WHERE year in ('2019','2020','2021')
	AND industry in (SELECT industry 
					 FROM top_industries)
GROUP BY industry, num_unicorns, year, average_valuation
ORDER BY industry, year DESC


,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
